In [1]:
pip install opencv-python mediapipe numpy pandas scikit-learn joblib


In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset path
DATASET_PATH = "C:\\Users\\Admin\\signdataset\\asl_alphabet_train"


# Load dataset using ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(DATASET_PATH, target_size=(64, 64),
                                         batch_size=16, class_mode="categorical", subset="training")

val_data = datagen.flow_from_directory(DATASET_PATH, target_size=(64, 64),
                                       batch_size=16, class_mode="categorical", subset="validation")

# Print class labels
print("Class labels:", train_data.class_indices)


Found 62400 images belonging to 26 classes.
Found 15600 images belonging to 26 classes.
Class labels: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_data, validation_data=val_data, epochs=10)

# Save model
model.save("sign_language_model.h5")
print("Model saved successfully!")


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1793s 459ms/step - accuracy: 0.4458 - loss: 1.8206 - val_accuracy: 0.6495 - val_loss: 1.0076
Epoch 2/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1812s 465ms/step - accuracy: 0.8732 - loss: 0.3628 - val_accuracy: 0.7179 - val_loss: 1.0649
Epoch 3/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1291s 331ms/step - accuracy: 0.9299 - loss: 0.1994 - val_accuracy: 0.7349 - val_loss: 0.9671
Epoch 4/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1083s 278ms/step - accuracy: 0.9493 - loss: 0.1443 - val_accuracy: 0.7561 - val_loss: 0.9271
Epoch 5/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1132s 290ms/step - accuracy: 0.9606 - loss: 0.1176 - val_accuracy: 0.7487 - val_loss: 1.0373
Epoch 6/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1036s 265ms/step - accuracy: 0.9669 - loss: 0.0981 - val_accuracy: 0.7428 - val_loss: 1.0736
Epoch 7/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1033s 265ms/step - accuracy: 0.9714 - loss: 0.0877 - val_accuracy: 0.7395 - val_loss: 1.1847
Epoch 8/10
3900/3900 ━━━━━━━━━━━━━━━━━━━━ 1030s 264ms/s

Model saved successfully!


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("sign_language_model.h5")

# Load class labels
labels = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
class_indices = {i: labels[i] for i in range(len(labels))}

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # Define ROI (Region of Interest)
    x, y, w, h = 100, 100, 300, 300
    roi = frame[y:y+h, x:x+w]
    
    # Preprocess image
    roi_resized = cv2.resize(roi, (64, 64))
    roi_resized = roi_resized / 255.0
    roi_resized = np.expand_dims(roi_resized, axis=0)

    # Predict sign
    prediction = model.predict(roi_resized)
    pred_index = np.argmax(prediction)
    pred_label = class_indices[pred_index]

    # Draw ROI
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(frame, f"Prediction: {pred_label}", (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display frame
    cv2.imshow("Sign Language Translator", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 